# Import Modules

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=50

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "LeafDisease",
    shuffle= True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 35961 files belonging to 4 classes.


2023-10-20 22:52:11.657158: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
for images, _ in dataset.take(1):  # Adjust the number of batches as needed
    min_value = tf.reduce_min(images).numpy()
    max_value = tf.reduce_max(images).numpy()
    print("Minimum value:", min_value)
    print("Maximum value:", max_value)

Minimum value: 0.0
Maximum value: 255.0


In [5]:
class_names = dataset.class_names
class_names

['Grape Black rot',
 'Grape Esca Black Measles',
 'Grape Leaf blight Isariopsis Leaf Spot',
 'Grape healthy']

In [6]:
len(dataset)

1124

In [7]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

(32, 256, 256, 3)
[2 2 1 2 0 0 2 0 0 0 0 1 0 2 3 0 0 0 0 1 1 1 0 2 0 1 3 1 1 1 1 1]


In [8]:
plt.figure(figsize=(15, 15))
for image_batch, labels_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")

In [9]:
train_size = 0.8
len(dataset)*train_size

899.2

In [10]:
train_ds = dataset.take(899)
len(train_ds)

899

In [11]:
test_ds = dataset.skip(899)
len(test_ds)

225

In [12]:
val_size=0.1
len(dataset)*val_size

112.4

In [13]:
val_ds = test_ds.take(112)
len(val_ds)

112

In [14]:
test_ds = test_ds.skip(112)
len(test_ds)


113

In [15]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [16]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [17]:
len(train_ds)

899

In [18]:
len(val_ds)

112

In [19]:
len(test_ds)

113

In [20]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [21]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [22]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [23]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [24]:
class Autoencoder(tf.keras.Model):
    def __init__(self):
        super().__init__()  # Correct super() call without arguments

        # N, 1, 28, 28
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Conv2D(16, (3, 3), strides=2, padding="same", activation="relu"),  # -> N, 16, 14, 14
            tf.keras.layers.Conv2D(32, (3, 3), strides=2, padding="same", activation="relu"),  # -> N, 32, 7, 7
            tf.keras.layers.Conv2D(64, (7, 7), activation="relu")  # -> N, 64, 1, 1
        ])

        # N, 64, 1, 1
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Conv2DTranspose(32, (7, 7), activation="relu"),  # -> N, 32, 7, 7
            tf.keras.layers.Conv2DTranspose(16, (3, 3), strides=2, padding="same", output_padding=1, activation="relu"),  # N, 16, 14, 14
            tf.keras.layers.Conv2DTranspose(1, (3, 3), strides=2, padding="same", output_padding=1, activation="sigmoid")  # N, 1, 28, 28
        ])

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded


In [25]:
model = Autoencoder()

# Define the loss function (Mean Squared Error)
loss_function = tf.keras.losses.MeanSquaredError()

# Define the optimizer (Adam optimizer)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)

In [8]:
num_epochs = 5
outputs = []

for epoch in range(num_epochs):
    x = 0
    for img, _ in dataset:
        with tf.GradientTape() as tape:
            recon = model(img)
            loss = tf.keras.losses.MeanSquaredError()(img, recon)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        x = x+1
        print(x)
    print(f'Epoch:{epoch+1}, Loss:{loss:.4f}')
    outputs.append((epoch, img, recon))


KeyboardInterrupt: 

In [ ]:
import numpy as np

for k in range(0, num_epochs, 4):
    plt.figure(figsize=(40, 30))
    plt.gray()
    imgs = outputs[k][1].numpy()
    recon = outputs[k][2].numpy()
    for i, item in enumerate(imgs):
        if i >= 3:
            break
        plt.subplot(2, 9, i + 1)
        # item: (height, width, channels)
        plt.imshow(item[:, :, 0], cmap='gray')
        
    for i, item in enumerate(recon):
        if i >= 3:
            break
        plt.subplot(2, 9, 9 + i + 1)  # row_length + i + 1
        # item: (height, width, channels)
        plt.imshow(item[:, :, 0], cmap='gray')



In [ ]:
for k in range(0, num_epochs, 4):
    plt.figure(figsize=(9, 2))
    recon = outputs[k][2].numpy()
    
    for i in range(9):
        plt.subplot(2, 9, i + 1)
        plt.imshow(recon[i, :, :], cmap='gray')
        plt.axis('off')  # Remove axis labels and ticks

plt.show()  # Display the reconstructed images
